In [ ]:
#This is run 3 folds, and 5 times for each data and each model.
#note that: do not save the model. 
#saved model is in next steps.

In [ ]:
# LSTM layers.
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
import pandas as pd
from numpy import array
from numpy import asarray
import numpy as np
from matplotlib import *
import matplotlib.pyplot as plt
#print(tf.__version__)
#fix with tf 2.1
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

import tensorflow as tf
import numpy as np
from numpy import array
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

def My_model(nodes, X_train, y_train,X_test, y_real,model_name):
    es = tf.keras.callbacks.EarlyStopping(
                    monitor='val_loss', patience=20,
                    restore_best_weights =True)
    #mc = tf.keras.callbacks.ModelCheckpoint(
    #                         filepath=model_name, 
    #                         monitor='val_loss', 
    #                         save_best_only=True)
    #the best model will be used, if you want to store the model, uncomment above.
    mc = tf.keras.callbacks.ModelCheckpoint(
                             filepath=model_name,
                             monitor='val_loss', 
                             save_best_only=True)
    callback = [es,mc]
    opt = tf.keras.optimizers.Adam(learning_rate=0.05)

    my_model = tf.keras.Sequential()
    my_model.add(tf.keras.layers.LSTM(units = nodes,input_shape = (X_train.shape[1],X_train.shape[2])))
    my_model.add(tf.keras.layers.Dropout(0.25))
    my_model.add(tf.keras.layers.Dense(1))
    my_model.compile(loss = 'mse',
     optimizer = opt, 
     metrics = ['mean_squared_error'])

    #my_model.summary()
    my_model.fit(X_train,y_train,
                    validation_data=(X_test, y_real),
                    batch_size = 500,epochs = 300,
                    callbacks=callback, verbose=0)
    return my_model
# source https://machinelearningmastery.com/convert-time-series-supervised-learning-problem-python/
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out-1
        # check if we are beyond the dataset
        if out_end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1:out_end_ix, -1]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

def Fill_data(df,mean,timesteps,n_features):
    columns = df.columns
    row_stand = df.iloc[0]
    for i in range (1,timesteps):
        row_temp = np.array([])
        for j in range(0,4):
            temp = row_stand[j]-i
            row_temp = np.append(row_temp,temp)
        for j in range(4,n_features):
            row_temp = np.append(row_temp,mean[j])

        minus = 0
        if(row_temp[0]<0): #index
            row_temp[0] = 0
        if(row_temp[1]<0): #hour
            row_temp[1] += 24
            minus = 1
        if(row_temp[2]<0 or minus == 1): #day of the year
            row_temp[2] += 365
            if(minus == 1):
                row_temp[2] -= 1
                minus= 0
        if(row_temp[3]<0): #weekday : from 0 to 6
            row_temp[4] = 0
        row_temp= pd.DataFrame(row_temp.reshape(1, len(row_temp)),columns=columns)
        df = pd.concat([row_temp,df],ignore_index=True,axis=0)
    return df

def Evaluation(my_model,X_test,y_real,scaler,timesteps,n_features):
    from numpy import concatenate
    from sklearn.metrics import mean_squared_error
    from math import sqrt
    y_pred = my_model.predict(X_test)

    X_test = X_test.reshape((X_test.shape[0], timesteps*(n_features-1)))
    #predict values
    from numpy import concatenate
    X_test = X_test[:, -(n_features-1):]
    y_pred = concatenate((X_test,y_pred), axis=1)
    y_pred = scaler.inverse_transform(y_pred)
    y_pred = y_pred[:,-1]

    y_real = concatenate((X_test,y_real), axis=1)
    y_real = scaler.inverse_transform(y_real)
    y_real = y_real[:,-1]

    RMSE = sqrt(mean_squared_error(y_real, y_pred))
    from sklearn.metrics import r2_score
    R2 = r2_score(y_real, y_pred)
    return RMSE,R2
def Plot_evaluation(my_model):
    #show evaluation early stop
    print(len(my_model.history.history['loss']))
    plt.plot(my_model.history.history['loss'], label='train')
    plt.plot(my_model.history.history['val_loss'], label='test')
    plt.legend()
    plt.show()


def Run(Train_path, Test_path):

    Train1 = pd.read_csv(Train_path)
    Test1 = pd.read_csv(Test_path)
    
    Train1 = Train1.dropna()
    Test1 = Test1.dropna()

    n_features=np.shape(Train1)[1] #12 inluding output

    print(Test1.shape)
    print(Train1.shape)
    columns = Train1.columns
    mean = Train1.mean(axis=0)


    List_nodes =[16,32,48,64,96,128]
    List_TimeSteps = [1,3,6,9,12]
    #nodes = 16
    #timesteps=1 
    for nodes in List_nodes:
        for timesteps in List_TimeSteps:

            #padding because of timesteps
            Train = Fill_data(Train1,mean,timesteps, n_features )
            Test  = Fill_data(Test1,mean,timesteps, n_features )

            #tranform train and test.
            New_data = [Train,Test]
            New_data = pd.concat(New_data)

            scaler = MinMaxScaler(feature_range=(0, 1))
            New_data = scaler.fit_transform(New_data)

            Train =New_data[0:Train.shape[0],:]
            Test = New_data[Train.shape[0]:,:]

            #print('LSTM nodes: ', nodes,' timesteps: ', timesteps)
            X_train, y_train = split_sequences(Train, timesteps, 1)
            X_test, y_real = split_sequences(Test, timesteps, 1)

            print( 'size after appling timesteps: ',X_train.shape,y_train.shape,X_test.shape, y_real.shape)


            #LSTM,
            my_model = My_model(nodes, X_train, y_train,X_test, y_real,
                                '3 folds_'+str(nodes)+'_'+str(timesteps)+'.h5')
            print('model stop after ',len(my_model.history.history['loss']), ' epochs')

            # Plot evaluation
            #Plot_evaluation(my_model)

            #Evaluate
            RMSE, R2 = Evaluation(my_model,X_test,y_real,scaler,timesteps,n_features)

            print('LSTM nodes: ', nodes,' timesteps: ', timesteps)
            print('RMSE: ', RMSE)
            print('R2: ', R2)
            print('\n\n')
            import os
            os.remove('3 folds_'+str(nodes)+'_'+str(timesteps)+'.h5')

In [ ]:

data_path= 'C:\\data\\New_data\\'

times = 20
for j in range(1, times+1):
    print('Time ', j)
    Run(data_path+'NO2_Train.csv',data_path+'NO2_Test.csv')

In [ ]:

    
data_path= 'C:\\data\\New_data\\'

times = 20
for j in range(1, times+1):
    print('Time ', j)
    Run(data_path+'CO_Train.csv',data_path+'CO_Test.csv')

In [ ]:


data_path= 'C:\\data\\New_data\\'

times = 20
for j in range(1, times+1):
    print('Time ', j)
    Run(data_path+'O3_Train.csv',data_path+'O3_Test.csv')

In [ ]:


data_path= 'C:\\data\\New_data\\'

times = 20
for j in range(1, times+1):
    print('Time ', j)
    Run(data_path+'PM2_5_Train.csv',data_path+'PM2_5_Test.csv')

In [ ]:


data_path= 'C:\\data\\New_data\\'

times = 20
for j in range(1, times+1):
    print('Time ', j)
    Run(data_path+'SO2_Train.csv',data_path+'SO2_Test.csv')

In [ ]:
#CNN
#CNN
# test with two, 3 LSTM layers.
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

#thebest is LSTM with 16 nodes - 1 layer
import tensorflow as tf
from numpy import array
from numpy import asarray
import numpy as np
from matplotlib import *
import matplotlib.pyplot as plt
#print(tf.__version__)
#fix with tf 2.1
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)


#process data... 
#Train from beginning to Index<140593
#validation: 140593<= Index < 142034
#from 142034 is quatifying data .. relaxation is from 143930

#LSTM 16- 1 layer - timestep =1
import tensorflow as tf
import numpy as np
import pandas as pd
from numpy import array
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

def My_model(nodes, X_train, y_train,X_test, y_real,kernel_size_, pool_size_,model_name):
    es = tf.keras.callbacks.EarlyStopping(
                    monitor='val_loss', patience=20,
                    restore_best_weights =True)
    mc = tf.keras.callbacks.ModelCheckpoint(
                             filepath=model_name, 
                             monitor='val_loss', 
                             save_best_only=True)
    #the best model will be used, if you want to store the model, uncomment above.

    callback = [es,mc]
    opt = tf.keras.optimizers.Adam(learning_rate=0.05)

    my_model = tf.keras.Sequential()
    my_model.add(tf.keras.layers.Conv1D(filters = nodes,
                                    kernel_size=kernel_size_,
                                    input_shape =
                                    (X_train.shape[1],
                                     X_train.shape[2])))
    my_model.add(tf.keras.layers.MaxPooling1D(pool_size=pool_size_))
    my_model.add(tf.keras.layers.Flatten())
    my_model.add(tf.keras.layers.Dropout(0.25))
    my_model.add(tf.keras.layers.Dense(1))

    my_model.compile(loss = 'mse',
     optimizer = opt, 
     metrics = ['mean_squared_error'])

    #my_model.summary()
    my_model.fit(X_train,y_train,
                    validation_data=(X_test, y_real),
                    batch_size = 500,epochs = 300,
                    callbacks=callback, verbose=0)
    return my_model
# source https://machinelearningmastery.com/convert-time-series-supervised-learning-problem-python/
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in):
	n_steps_out=1
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps_in
		out_end_ix = end_ix
		# check if we are beyond the dataset
		if out_end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seqy = sequences[i:end_ix, -n_steps_out:]
		seq_x, seq_y = sequences[i:end_ix, :-1], seqy[-1,:]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

def Fill_data(df,mean,timesteps,n_features):
    columns = df.columns
    row_stand = df.iloc[0]
    for i in range (1,timesteps):
        row_temp = np.array([])
        for j in range(0,4):
            temp = row_stand[j]-i
            row_temp = np.append(row_temp,temp)
        for j in range(4,n_features):
            row_temp = np.append(row_temp,mean[j])

        minus = 0
        if(row_temp[0]<0): #index
            row_temp[0] = 0
        if(row_temp[1]<0): #hour
            row_temp[1] += 24
            minus = 1
        if(row_temp[2]<0 or minus == 1): #day of the year
            row_temp[2] += 365
            if(minus == 1):
                row_temp[2] -= 1
                minus= 0
        if(row_temp[3]<0): #weekday : from 0 to 6
            row_temp[4] = 0
        row_temp= pd.DataFrame(row_temp.reshape(1, len(row_temp)),columns=columns)
        df = pd.concat([row_temp,df],ignore_index=True,axis=0)
    return df

def Evaluation(my_model,X_test,y_real,scaler,timesteps,n_features):
    from numpy import concatenate
    from sklearn.metrics import mean_squared_error
    from math import sqrt
    y_pred = my_model.predict(X_test)

    X_test = X_test.reshape((X_test.shape[0], timesteps*(n_features-1)))
    #predict values
    from numpy import concatenate
    X_test = X_test[:, -(n_features-1):]
    y_pred = concatenate((X_test,y_pred), axis=1)
    y_pred = scaler.inverse_transform(y_pred)
    y_pred = y_pred[:,-1]

    y_real = concatenate((X_test,y_real), axis=1)
    y_real = scaler.inverse_transform(y_real)
    y_real = y_real[:,-1]

    RMSE = sqrt(mean_squared_error(y_real, y_pred))
    from sklearn.metrics import r2_score
    R2 = r2_score(y_real, y_pred)
    return RMSE,R2
def Plot_evaluation(my_model):
    #show evaluation early stop
    print(len(my_model.history.history['loss']))
    plt.plot(my_model.history.history['loss'], label='train')
    plt.plot(my_model.history.history['val_loss'], label='test')
    plt.legend()
    plt.show()
#load train and test data here
##process data... 
#Train from beginning to Index<140593
#validation: 140593<= Index < 142034
#from 142034 is quatifying data .. relaxation is from 143930

def Run(Train_path, Test_path):

    Train1 = pd.read_csv(Train_path)
    Test1 = pd.read_csv(Test_path)
    
    Train1 = Train1.dropna()
    Test1 = Test1.dropna()

    n_features=np.shape(Train1)[1] #12 inluding output

    print(Test1.shape)
    print(Train1.shape)
    columns = Train1.columns
    mean = Train1.mean(axis=0)


    List_nodes =[16,32,48,64,96,128]
    List_TimeSteps = [1,3,6,9,12]
    #nodes = 16
    #timesteps=1 
    for nodes in List_nodes:
        for timesteps in List_TimeSteps:
            kernel_size_= 3
            pool_size_ = 3
            if(timesteps <= kernel_size_):
                print('warning: stepsizes is less or equal than kernel size' )
                kernel_size_=max(1,timesteps-1)
                pool_size_ =max(1,timesteps-1)
            #padding because of timesteps
            Train = Fill_data(Train1,mean,timesteps, n_features )
            Test  = Fill_data(Test1,mean,timesteps, n_features )

            #tranform train and test.
            New_data = [Train,Test]
            New_data = pd.concat(New_data)

            scaler = MinMaxScaler(feature_range=(0, 1))
            New_data = scaler.fit_transform(New_data)

            Train =New_data[0:Train.shape[0],:]
            Test = New_data[Train.shape[0]:,:]

            #print('LSTM nodes: ', nodes,' timesteps: ', timesteps)
            X_train, y_train = split_sequences(Train, timesteps)
            X_test, y_real = split_sequences(Test, timesteps)

            print( 'size after appling timesteps: ',X_train.shape,y_train.shape,X_test.shape, y_real.shape)


            my_model = My_model(nodes, X_train, y_train,X_test, y_real,kernel_size_, pool_size_,'3 folds_'+str(nodes)+'_'+str(timesteps)+'.h5')
            print('model stop after ',len(my_model.history.history['loss']), ' epochs')

            # Plot evaluation
            #Plot_evaluation(my_model)

            #Evaluate
            RMSE, R2 = Evaluation(my_model,X_test,y_real,scaler,timesteps,n_features)

            print('CNN nodes: ', nodes,' timesteps: ', timesteps)
            print('RMSE: ', RMSE)
            print('R2: ', R2)
            print('\n\n')
            import os
            os.remove('3 folds_'+str(nodes)+'_'+str(timesteps)+'.h5')

In [ ]:
data_path= 'C:\\data\\New_data\\'

times = 20
for j in range(1, times+1):
    print('Time ', j)
    Run(data_path+'NO2_Train.csv',data_path+'NO2_Test.csv')


In [ ]:

    
data_path= 'C:\\data\\New_data\\'

times = 20
for j in range(1, times+1):
    print('Time ', j)
    Run(data_path+'CO_Train.csv',data_path+'CO_Test.csv')



In [ ]:

data_path= 'C:\\data\\New_data\\'

times = 20
for j in range(1, times+1):
    print('Time ', j)
    Run(data_path+'O3_Train.csv',data_path+'O3_Test.csv')


In [ ]:


data_path= 'C:\\data\\New_data\\'

times = 20
for j in range(1, times+1):
    print('Time ', j)
    Run(data_path+'PM2_5_Train.csv',data_path+'PM2_5_Test.csv')



In [ ]:


data_path= 'C:\\data\\New_data\\'

times = 20
for j in range(1, times+1):
    print('Time ', j)
    Run(data_path+'SO2_Train.csv',data_path+'SO2_Test.csv')

In [ ]:
#SimpleRNN
# test with two, 3 LSTM layers.
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

#thebest is LSTM with 16 nodes - 1 layer
import tensorflow as tf
import pandas as pd
from numpy import array
from numpy import asarray
import numpy as np
from matplotlib import *
import matplotlib.pyplot as plt
#print(tf.__version__)
#fix with tf 2.1
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)


#process data... 
#Train from beginning to Index<140593
#validation: 140593<= Index < 142034
#from 142034 is quatifying data .. relaxation is from 143930

#LSTM 16- 1 layer - timestep =1
import tensorflow as tf
import numpy as np
from numpy import array
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

def My_model(nodes, X_train, y_train,X_test, y_real,model_name ):
    es = tf.keras.callbacks.EarlyStopping(
                    monitor='val_loss', patience=20,
                    restore_best_weights =True)
    mc = tf.keras.callbacks.ModelCheckpoint(
                             filepath=model_name, 
                             monitor='val_loss', 
                             save_best_only=True)
    #the best model will be used, if you want to store the model, uncomment above.
    #mc = tf.keras.callbacks.ModelCheckpoint(
    #                         monitor='val_loss', 
    #                         save_best_only=True)
    callback = [es,mc]
    opt = tf.keras.optimizers.Adam(learning_rate=0.05)
 
    my_model = tf.keras.Sequential()
    my_model.add( tf.keras.layers.SimpleRNN(
                            units = nodes,
                            input_shape = (X_train.shape[1],X_train.shape[2])))#return_state=True)) #return_sequences=True, 
    my_model.add(tf.keras.layers.Dropout(0.25))
    my_model.add(tf.keras.layers.Dense(1))
    
    my_model.compile(loss = 'mse',
     optimizer = opt, 
     metrics = ['mean_squared_error'])

    #my_model.summary()
    my_model.fit(X_train,y_train,
                    validation_data=(X_test, y_real),
                    batch_size = 500,epochs = 300,
                    callbacks=callback, verbose=0)
    return my_model
# source https://machinelearningmastery.com/convert-time-series-supervised-learning-problem-python/
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out-1
        # check if we are beyond the dataset
        if out_end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1:out_end_ix, -1]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

def Fill_data(df,mean,timesteps,n_features):
    columns = df.columns
    row_stand = df.iloc[0]
    for i in range (1,timesteps):
        row_temp = np.array([])
        for j in range(0,4):
            temp = row_stand[j]-i
            row_temp = np.append(row_temp,temp)
        for j in range(4,n_features):
            row_temp = np.append(row_temp,mean[j])

        minus = 0
        if(row_temp[0]<0): #index
            row_temp[0] = 0
        if(row_temp[1]<0): #hour
            row_temp[1] += 24
            minus = 1
        if(row_temp[2]<0 or minus == 1): #day of the year
            row_temp[2] += 365
            if(minus == 1):
                row_temp[2] -= 1
                minus= 0
        if(row_temp[3]<0): #weekday : from 0 to 6
            row_temp[4] = 0
        row_temp= pd.DataFrame(row_temp.reshape(1, len(row_temp)),columns=columns)
        df = pd.concat([row_temp,df],ignore_index=True,axis=0)
    return df

def Evaluation(my_model,X_test,y_real,scaler,timesteps,n_features):
    from numpy import concatenate
    from sklearn.metrics import mean_squared_error
    from math import sqrt
    y_pred = my_model.predict(X_test)

    X_test = X_test.reshape((X_test.shape[0], timesteps*(n_features-1)))
    #predict values
    from numpy import concatenate
    X_test = X_test[:, -(n_features-1):]
    y_pred = concatenate((X_test,y_pred), axis=1)
    y_pred = scaler.inverse_transform(y_pred)
    y_pred = y_pred[:,-1]

    y_real = concatenate((X_test,y_real), axis=1)
    y_real = scaler.inverse_transform(y_real)
    y_real = y_real[:,-1]

    RMSE = sqrt(mean_squared_error(y_real, y_pred))
    from sklearn.metrics import r2_score
    R2 = r2_score(y_real, y_pred)
    return RMSE,R2
def Plot_evaluation(my_model):
    #show evaluation early stop
    print(len(my_model.history.history['loss']))
    plt.plot(my_model.history.history['loss'], label='train')
    plt.plot(my_model.history.history['val_loss'], label='test')
    plt.legend()
    plt.show()
#load train and test data here
##process data... 
#Train from beginning to Index<140593
#validation: 140593<= Index < 142034
#from 142034 is quatifying data .. relaxation is from 143930

def Run(Train_path, Test_path):

    Train1 = pd.read_csv(Train_path)
    Test1 = pd.read_csv(Test_path)
    
    Train1 = Train1.dropna()
    Test1 = Test1.dropna()

    n_features=np.shape(Train1)[1] #12 inluding output

    print(Test1.shape)
    print(Train1.shape)
    columns = Train1.columns
    mean = Train1.mean(axis=0)


    List_nodes =[16,32,48,64,96,128]
    List_TimeSteps = [1,3,6,9,12]
    #nodes = 16
    #timesteps=1 
    for nodes in List_nodes:
        for timesteps in List_TimeSteps:

            #padding because of timesteps
            Train = Fill_data(Train1,mean,timesteps, n_features )
            Test  = Fill_data(Test1,mean,timesteps, n_features )

            #tranform train and test.
            New_data = [Train,Test]
            New_data = pd.concat(New_data)

            scaler = MinMaxScaler(feature_range=(0, 1))
            New_data = scaler.fit_transform(New_data)

            Train =New_data[0:Train.shape[0],:]
            Test = New_data[Train.shape[0]:,:]

            #print('LSTM nodes: ', nodes,' timesteps: ', timesteps)
            X_train, y_train = split_sequences(Train, timesteps, 1)
            X_test, y_real = split_sequences(Test, timesteps, 1)

            print( 'size after appling timesteps: ',X_train.shape,y_train.shape,X_test.shape, y_real.shape)


            #LSTM,
            my_model = My_model(nodes, X_train, y_train,X_test, y_real,'3 folds_'+str(nodes)+'_'+str(timesteps)+'.h5')
            print('model stop after ',len(my_model.history.history['loss']), ' epochs')

            # Plot evaluation
            #Plot_evaluation(my_model)

            #Evaluate
            RMSE, R2 = Evaluation(my_model,X_test,y_real,scaler,timesteps,n_features)

            print('SimpleRNN nodes: ', nodes,' timesteps: ', timesteps)
            print('RMSE: ', RMSE)
            print('R2: ', R2)
            print('\n\n')
            import os
            os.remove('3 folds_'+str(nodes)+'_'+str(timesteps)+'.h5')


In [ ]:
data_path= 'C:\\data\\New_data\\'

times = 20
for j in range(1, times+1):
    print('Time ', j)
    Run(data_path+'NO2_Train.csv',data_path+'NO2_Test.csv')
    

In [ ]:

    
data_path= 'C:\\data\\New_data\\'

times = 20
for j in range(1, times+1):
    print('Time ', j)
    Run(data_path+'CO_Train.csv',data_path+'CO_Test.csv')



In [ ]:


data_path= 'C:\\data\\New_data\\'

times = 20
for j in range(1, times+1):
    print('Time ', j)
    Run(data_path+'O3_Train.csv',data_path+'O3_Test.csv')



In [ ]:


data_path= 'C:\\data\\New_data\\'

times = 20
for j in range(1, times+1):
    print('Time ', j)
    Run(data_path+'PM2_5_Train.csv',data_path+'PM2_5_Test.csv')


In [ ]:

data_path= 'C:\\data\\New_data\\'

times = 20
for j in range(1, times+1):
    print('Time ', j)
    Run(data_path+'SO2_Train.csv',data_path+'SO2_Test.csv')

In [ ]:

#BiRNN
# test with two, 3 LSTM layers.
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

#thebest is LSTM with 16 nodes - 1 layer
import tensorflow as tf
import pandas as pd
from numpy import array
from numpy import asarray
import numpy as np
from matplotlib import *
import matplotlib.pyplot as plt
#print(tf.__version__)
#fix with tf 2.1
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)


#process data... 
#Train from beginning to Index<140593
#validation: 140593<= Index < 142034
#from 142034 is quatifying data .. relaxation is from 143930

#LSTM 16- 1 layer - timestep =1
import tensorflow as tf
import numpy as np
from numpy import array
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

def My_model(nodes, X_train, y_train,X_test, y_real,model_name ):
    es = tf.keras.callbacks.EarlyStopping(
                    monitor='val_loss', patience=20,
                    restore_best_weights =True)
    mc = tf.keras.callbacks.ModelCheckpoint(
                             filepath=model_name, 
                             monitor='val_loss', 
                             save_best_only=True)
    #the best model will be used, if you want to store the model, uncomment above.
    #mc = tf.keras.callbacks.ModelCheckpoint(
    #                         monitor='val_loss', 
    #                         save_best_only=True)
    callback = [es,mc]
    opt = tf.keras.optimizers.Adam(learning_rate=0.05)
    
    my_model = tf.keras.Sequential()
    my_model.add(tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(
            units = nodes,
            input_shape = (X_train.shape[1],X_train.shape[2])))
        )
    my_model.add(tf.keras.layers.Dropout(0.25))
    my_model.add(tf.keras.layers.Dense(1))
    
    my_model.compile(loss = 'mse',
     optimizer = opt, 
     metrics = ['mean_squared_error'])

    #my_model.summary()
    my_model.fit(X_train,y_train,
                    validation_data=(X_test, y_real),
                    batch_size = 500,epochs = 300,
                    callbacks=callback, verbose=0)
    return my_model
# source https://machinelearningmastery.com/convert-time-series-supervised-learning-problem-python/
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out-1
        # check if we are beyond the dataset
        if out_end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1:out_end_ix, -1]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

def Fill_data(df,mean,timesteps,n_features):
    columns = df.columns
    row_stand = df.iloc[0]
    for i in range (1,timesteps):
        row_temp = np.array([])
        for j in range(0,4):
            temp = row_stand[j]-i
            row_temp = np.append(row_temp,temp)
        for j in range(4,n_features):
            row_temp = np.append(row_temp,mean[j])

        minus = 0
        if(row_temp[0]<0): #index
            row_temp[0] = 0
        if(row_temp[1]<0): #hour
            row_temp[1] += 24
            minus = 1
        if(row_temp[2]<0 or minus == 1): #day of the year
            row_temp[2] += 365
            if(minus == 1):
                row_temp[2] -= 1
                minus= 0
        if(row_temp[3]<0): #weekday : from 0 to 6
            row_temp[4] = 0
        row_temp= pd.DataFrame(row_temp.reshape(1, len(row_temp)),columns=columns)
        df = pd.concat([row_temp,df],ignore_index=True,axis=0)
    return df

def Evaluation(my_model,X_test,y_real,scaler,timesteps,n_features):
    from numpy import concatenate
    from sklearn.metrics import mean_squared_error
    from math import sqrt
    y_pred = my_model.predict(X_test)

    X_test = X_test.reshape((X_test.shape[0], timesteps*(n_features-1)))
    #predict values
    from numpy import concatenate
    X_test = X_test[:, -(n_features-1):]
    y_pred = concatenate((X_test,y_pred), axis=1)
    y_pred = scaler.inverse_transform(y_pred)
    y_pred = y_pred[:,-1]

    y_real = concatenate((X_test,y_real), axis=1)
    y_real = scaler.inverse_transform(y_real)
    y_real = y_real[:,-1]

    RMSE = sqrt(mean_squared_error(y_real, y_pred))
    from sklearn.metrics import r2_score
    R2 = r2_score(y_real, y_pred)
    return RMSE,R2
def Plot_evaluation(my_model):
    #show evaluation early stop
    print(len(my_model.history.history['loss']))
    plt.plot(my_model.history.history['loss'], label='train')
    plt.plot(my_model.history.history['val_loss'], label='test')
    plt.legend()
    plt.show()
#load train and test data here
##process data... 
#Train from beginning to Index<140593
#validation: 140593<= Index < 142034
#from 142034 is quatifying data .. relaxation is from 143930

def Run(Train_path, Test_path):

    Train1 = pd.read_csv(Train_path)
    Test1 = pd.read_csv(Test_path)
    
    Train1 = Train1.dropna()
    Test1 = Test1.dropna()

    n_features=np.shape(Train1)[1] #12 inluding output

    print(Test1.shape)
    print(Train1.shape)
    columns = Train1.columns
    mean = Train1.mean(axis=0)


    List_nodes =[16,32,48,64,96,128]
    List_TimeSteps = [1,3,6,9,12]
    #nodes = 16
    #timesteps=1 
    for nodes in List_nodes:
        for timesteps in List_TimeSteps:

            #padding because of timesteps
            Train = Fill_data(Train1,mean,timesteps, n_features )
            Test  = Fill_data(Test1,mean,timesteps, n_features )

            #tranform train and test.
            New_data = [Train,Test]
            New_data = pd.concat(New_data)

            scaler = MinMaxScaler(feature_range=(0, 1))
            New_data = scaler.fit_transform(New_data)

            Train =New_data[0:Train.shape[0],:]
            Test = New_data[Train.shape[0]:,:]

            #print('BiRNN nodes: ', nodes,' timesteps: ', timesteps)
            X_train, y_train = split_sequences(Train, timesteps, 1)
            X_test, y_real = split_sequences(Test, timesteps, 1)

            print( 'size after appling timesteps: ',X_train.shape,y_train.shape,X_test.shape, y_real.shape)


            #BiRNN,
            my_model = My_model(nodes, X_train, y_train,X_test, y_real,'3 folds_'+str(nodes)+'_'+str(timesteps)+'.h5')
            print('model stop after ',len(my_model.history.history['loss']), ' epochs')

            # Plot evaluation
            #Plot_evaluation(my_model)

            #Evaluate
            RMSE, R2 = Evaluation(my_model,X_test,y_real,scaler,timesteps,n_features)

            print('BiRNN nodes: ', nodes,' timesteps: ', timesteps)
            print('RMSE: ', RMSE)
            print('R2: ', R2)
            print('\n\n')
            
            import os
            os.remove('3 folds_'+str(nodes)+'_'+str(timesteps)+'.h5')


In [ ]:
data_path= 'C:\\data\\New_data\\'

times = 20
for j in range(1, times+1):
    print('Time ', j)
    Run(data_path+'NO2_Train.csv',data_path+'NO2_Test.csv')
 

In [ ]:


times = 20
for j in range(1, times+1):
    print('Time ', j)
    Run(data_path+'CO_Train.csv',data_path+'CO_Test.csv')


In [ ]:


data_path= 'C:\\data\\New_data\\'

times = 20
for j in range(1, times+1):
    print('Time ', j)
    Run(data_path+'O3_Train.csv',data_path+'O3_Test.csv')


In [ ]:


data_path= 'C:\\data\\New_data\\'

times = 20
for j in range(1, times+1):
    print('Time ', j)
    Run(data_path+'PM2_5_Train.csv',data_path+'PM2_5_Test.csv')


In [ ]:


data_path= 'C:\\data\\New_data\\'

times = 20
for j in range(1, times+1):
    print('Time ', j)
    Run(data_path+'SO2_Train.csv',data_path+'SO2_Test.csv')

In [ ]:

#GRU
# test with two, 3 LSTM layers.
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

#thebest is LSTM with 16 nodes - 1 layer
import tensorflow as tf
import pandas as pd
from numpy import array
from numpy import asarray
import numpy as np
from matplotlib import *
import matplotlib.pyplot as plt
#print(tf.__version__)
#fix with tf 2.1
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)


#process data... 
#Train from beginning to Index<140593
#validation: 140593<= Index < 142034
#from 142034 is quatifying data .. relaxation is from 143930

#LSTM 16- 1 layer - timestep =1
import tensorflow as tf
import numpy as np
from numpy import array
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

def My_model(nodes, X_train, y_train,X_test, y_real,model_name ):
    es = tf.keras.callbacks.EarlyStopping(
                    monitor='val_loss', patience=20,
                    restore_best_weights =True)
    mc = tf.keras.callbacks.ModelCheckpoint(
                             filepath=model_name, 
                             monitor='val_loss', 
                             save_best_only=True)
    #the best model will be used, if you want to store the model, uncomment above.
    #mc = tf.keras.callbacks.ModelCheckpoint(
    #                         monitor='val_loss', 
    #                         save_best_only=True)
    callback = [es,mc]
    opt = tf.keras.optimizers.Adam(learning_rate=0.05)


    my_model = tf.keras.Sequential()
    my_model.add(
                tf.keras.layers.GRU(
                units = nodes,
                input_shape = (X_train.shape[1],X_train.shape[2])))#return_state=True)) #return_sequences=True, 
    my_model.add(tf.keras.layers.Dropout(0.25))
    my_model.add(tf.keras.layers.Dense(1))
    
    my_model.compile(loss = 'mse',
     optimizer = opt, 
     metrics = ['mean_squared_error'])

    #my_model.summary()
    my_model.fit(X_train,y_train,
                    validation_data=(X_test, y_real),
                    batch_size = 500,epochs = 300,
                    callbacks=callback, verbose=0)
    return my_model
# source https://machinelearningmastery.com/convert-time-series-supervised-learning-problem-python/
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out-1
        # check if we are beyond the dataset
        if out_end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1:out_end_ix, -1]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

def Fill_data(df,mean,timesteps,n_features):
    columns = df.columns
    row_stand = df.iloc[0]
    for i in range (1,timesteps):
        row_temp = np.array([])
        for j in range(0,4):
            temp = row_stand[j]-i
            row_temp = np.append(row_temp,temp)
        for j in range(4,n_features):
            row_temp = np.append(row_temp,mean[j])

        minus = 0
        if(row_temp[0]<0): #index
            row_temp[0] = 0
        if(row_temp[1]<0): #hour
            row_temp[1] += 24
            minus = 1
        if(row_temp[2]<0 or minus == 1): #day of the year
            row_temp[2] += 365
            if(minus == 1):
                row_temp[2] -= 1
                minus= 0
        if(row_temp[3]<0): #weekday : from 0 to 6
            row_temp[4] = 0
        row_temp= pd.DataFrame(row_temp.reshape(1, len(row_temp)),columns=columns)
        df = pd.concat([row_temp,df],ignore_index=True,axis=0)
    return df

def Evaluation(my_model,X_test,y_real,scaler,timesteps,n_features):
    from numpy import concatenate
    from sklearn.metrics import mean_squared_error
    from math import sqrt
    y_pred = my_model.predict(X_test)

    X_test = X_test.reshape((X_test.shape[0], timesteps*(n_features-1)))
    #predict values
    from numpy import concatenate
    X_test = X_test[:, -(n_features-1):]
    y_pred = concatenate((X_test,y_pred), axis=1)
    y_pred = scaler.inverse_transform(y_pred)
    y_pred = y_pred[:,-1]

    y_real = concatenate((X_test,y_real), axis=1)
    y_real = scaler.inverse_transform(y_real)
    y_real = y_real[:,-1]

    RMSE = sqrt(mean_squared_error(y_real, y_pred))
    from sklearn.metrics import r2_score
    R2 = r2_score(y_real, y_pred)
    return RMSE,R2
def Plot_evaluation(my_model):
    #show evaluation early stop
    print(len(my_model.history.history['loss']))
    plt.plot(my_model.history.history['loss'], label='train')
    plt.plot(my_model.history.history['val_loss'], label='test')
    plt.legend()
    plt.show()
#load train and test data here
##process data... 
#Train from beginning to Index<140593
#validation: 140593<= Index < 142034
#from 142034 is quatifying data .. relaxation is from 143930

def Run(Train_path, Test_path):

    Train1 = pd.read_csv(Train_path)
    Test1 = pd.read_csv(Test_path)
    
    Train1 = Train1.dropna()
    Test1 = Test1.dropna()

    n_features=np.shape(Train1)[1] #12 inluding output

    print(Test1.shape)
    print(Train1.shape)
    columns = Train1.columns
    mean = Train1.mean(axis=0)


    List_nodes =[16,32,48,64,96,128]
    List_TimeSteps = [1,3,6,9,12]
    #nodes = 16
    #timesteps=1 
    for nodes in List_nodes:
        for timesteps in List_TimeSteps:

            #padding because of timesteps
            Train = Fill_data(Train1,mean,timesteps, n_features )
            Test  = Fill_data(Test1,mean,timesteps, n_features )

            #tranform train and test.
            New_data = [Train,Test]
            New_data = pd.concat(New_data)

            scaler = MinMaxScaler(feature_range=(0, 1))
            New_data = scaler.fit_transform(New_data)

            Train =New_data[0:Train.shape[0],:]
            Test = New_data[Train.shape[0]:,:]

            #print('LSTM nodes: ', nodes,' timesteps: ', timesteps)
            X_train, y_train = split_sequences(Train, timesteps, 1)
            X_test, y_real = split_sequences(Test, timesteps, 1)

            print( 'size after appling timesteps: ',X_train.shape,y_train.shape,X_test.shape, y_real.shape)


            #LSTM,
            my_model = My_model(nodes, X_train, y_train,X_test, y_real,'3 folds_'+str(nodes)+'_'+str(timesteps)+'.h5')
            print('model stop after ',len(my_model.history.history['loss']), ' epochs')

            # Plot evaluation
            #Plot_evaluation(my_model)

            #Evaluate
            RMSE, R2 = Evaluation(my_model,X_test,y_real,scaler,timesteps,n_features)

            print('GRU nodes: ', nodes,' timesteps: ', timesteps)
            print('RMSE: ', RMSE)
            print('R2: ', R2)
            print('\n\n')
            
            import os
            os.remove('3 folds_'+str(nodes)+'_'+str(timesteps)+'.h5')


In [ ]:
data_path= 'C:\\data\\New_data\\'

times = 20
for j in range(1, times+1):
    print('Time ', j)
    Run(data_path+'NO2_Train.csv',data_path+'NO2_Test.csv')


In [ ]:

    
data_path= 'C:\\data\\New_data\\'

times = 20
for j in range(1, times+1):
    print('Time ', j)
    Run(data_path+'CO_Train.csv',data_path+'CO_Test.csv')


In [ ]:


data_path= 'C:\\data\\New_data\\'

times = 20
for j in range(1, times+1):
    print('Time ', j)
    Run(data_path+'O3_Train.csv',data_path+'O3_Test.csv')


In [ ]:

data_path= 'C:\\data\\New_data\\'

times = 20
for j in range(1, times+1):
    print('Time ', j)
    Run(data_path+'PM2_5_Train.csv',data_path+'PM2_5_Test.csv')


In [ ]:
data_path= 'C:\\data\\New_data\\'

times = 20
for j in range(1, times+1):
    print('Time ', j)
    Run(data_path+'SO2_Train.csv',data_path+'SO2_Test.csv')